In [1]:
import os

In [2]:
%pwd

'e:\\House-Price-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\House-Price-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.data_root])
        
    def get_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        
        return data_transformation_config
        
        

In [8]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [13]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def clean_data(self, data):
        # Perform data cleaning operations here
        cleaned_data = data.dropna(subset=['total_bedrooms'])
        return cleaned_data

    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)
        
        # data cleaning(drop all rows with missing values)
        cleaned_data = self.clean_data(data)
        
        # split the data into training and testing sets, (0.9,0.1) splits.
        train, test = train_test_split(cleaned_data, test_size=0.1)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index = False)
        
        logger.info("splitted data into training and testing sets")
        logger.info(train.shape)
        logger.info(test.shape)
        
        print(train.shape)
        print(test.shape)
        

In [14]:
try:
    config = ConfigManager()
    data_transformation_config = config.get_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2024-07-05 22:53:50,523: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-05 22:53:50,525: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-05 22:53:50,530: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-05 22:53:50,533: INFO: common: created directory at: data]
[2024-07-05 22:53:50,534: INFO: common: created directory at: data/data_transformation]
[2024-07-05 22:53:50,715: INFO: 3870339948: splitted data into training and testing sets]
[2024-07-05 22:53:50,715: INFO: 3870339948: (18389, 10)]
[2024-07-05 22:53:50,716: INFO: 3870339948: (2044, 10)]
(18389, 10)
(2044, 10)
